In [1]:
import os
import pandas as pd
from tqdm import tqdm 
from sentence_transformers import SentenceTransformer
import time
from lxml import etree

In [2]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
def calculate_embedding(text_list):
    return [model.encode(text) for text in text_list]

def extract_heading_text(xml_content):
    try:
        # Parsear el contenido XML para crear un árbol de elementos
        root = etree.fromstring(xml_content)
        
        # Lista para guardar textos de las etiquetas 'heading'
        heading_texts = []        
        # Recorrer todos los elementos y extraer textos de las etiquetas 'heading'
        for element in root.iter('heading'):
            # Agregar el texto de la etiqueta 'heading' a la lista
            heading_texts.append(element.text.strip() if element.text else "")
        
        return heading_texts
    except etree.XMLSyntaxError:
        print("Error al parsear xml")
        return []  
    
def extract_all_xml_tags(xml_content):
    try:
        # Parsear el contenido XML para crear un árbol de elementos
        root = etree.fromstring(xml_content)
        # Inicializar una lista para guardar solo los nombres de las etiquetas
        all_tags = []
        # Recorrer todos los elementos en el árbol XML
        for element in root.iter():
            all_tags.append(element.tag)
    
        return all_tags
    
    except etree.XMLSyntaxError:
        print("Error al procesar")
        return []

In [4]:
from lxml import etree
def parse_xml_from_bytes(data_bytes):
    try:
        # Parsear el XML directamente desde bytes
        root = etree.fromstring(data_bytes)
        # Para extraer todo el texto concatenado de los elementos XML
        return ''.join(root.xpath('//text()'))
    except Exception as e:
        print(f"Error al parsear XML: {e}")
        return None


In [ ]:
# Ruta donde se encuentran los archivos Parquet originales a leer
path = '/export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated'
# Ruta donde se guardarán los archivos Parquet procesados
output_path = '/export/data_ml4ds/NextProcurement/PLACE/BSC_procesados_ca'

# Crear la lista de todos los archivos .parq en el directorio especificado
all_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.parq')]

palabras_clave = ['tecnicas']
# Expresión regular para identificar palabra clave
regex = '|'.join(palabras_clave)

# Usar tqdm en el bucle externo para ver el progreso en el procesamiento de archivos
for file in tqdm(all_files, desc="Procesando archivos"):
    # Leer el archivo Parquet
    df = pd.read_parquet(file)

    start_time = time.time()
    # Filtrado del df para filas donde la columna 'doc_name' contenga palabra clave
    df = df[df['doc_name'].str.contains(regex, na=False, case=False)]
    # Añadir una columna para identificar secciones del xml y meterlo en una columna llamada secciones.
    df['extracted_tags'] = df['content'].apply(extract_all_xml_tags)
    df['texto_heading'] = df['content'].apply(extract_heading_text)
    df['extracted'] = df['content'].apply(parse_xml_from_bytes)
    # Filtrar las filas donde la lista 'texto_heading' está vacía
    df = df[df['texto_heading'].map(len) > 0]

    # Usar tqdm en el apply para ver el progreso del cálculo de embeddings
    #tqdm.pandas(desc="Calculando embeddings")
    #df['embeddings_heading'] = df['texto_heading'].progress_apply(calculate_embedding)
    print(df.columns)
    df = df[['procurement_id', 'doc_name', 'extracted', 'extracted_tags', 'texto_heading', "translated_content"]]#'embeddings_heading'

    end_time = time.time()
    print(f"Archivo procesado: {file} en {end_time - start_time} segundos")
    
    # Construir el nombre del nuevo archivo conservando el nombre original pero cambiando la ruta
    new_file_path = os.path.join(output_path, os.path.basename(file))
    # Guardar el DataFrame en la nueva ruta
    df.to_parquet(new_file_path)

Procesando archivos:   0%|          | 0/17 [00:00<?, ?it/s]

Index(['procurement_id', 'doc_name', 'content', 'alternative_lang',
       'translated_content', 'extracted_tags', 'texto_heading', 'extracted'],
      dtype='object')
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_15_containing_252064_docs_mark_fixed.parq en 82.64174365997314 segundos


Procesando archivos:   6%|▌         | 1/17 [02:10<34:44, 130.26s/it]

Index(['procurement_id', 'doc_name', 'content', 'alternative_lang',
       'translated_content', 'extracted_tags', 'texto_heading', 'extracted'],
      dtype='object')
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_2_containing_252000_docs_mark_fixed.parq en 81.29951477050781 segundos


Procesando archivos:  12%|█▏        | 2/17 [04:20<32:30, 130.01s/it]

...

Procesando archivos:  41%|████      | 7/17 [17:11<25:10, 151.10s/it]

Index(['procurement_id', 'doc_name', 'content', 'alternative_lang',
       'translated_content', 'extracted_tags', 'texto_heading', 'extracted'],
      dtype='object')
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_8_containing_251687_docs_mark_fixed.parq en 78.07225012779236 segundos


Procesando archivos:  47%|████▋     | 8/17 [19:46<22:49, 152.14s/it]

Index(['procurement_id', 'doc_name', 'content', 'alternative_lang',
       'translated_content', 'extracted_tags', 'texto_heading', 'extracted'],
      dtype='object')
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/pdf2txt_ca_translated/procurements_file_1_containing_251067_docs_mark_fixed.parq en 77.60586380958557 segundos


Procesando archivos:  53%|█████▎    | 9/17 [22:20<20:21, 152.72s/it]